<a href="https://colab.research.google.com/github/kbpotts1/ThinkfulProjects/blob/master/K_Potts_Thinkful_Final_Capstone_Project_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf

In [0]:
import keras

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split

from keras.models import Model, load_model, Sequential
from keras.layers import Input, Dense, Dropout
from keras.callbacks import ModelCheckpoint, TensorBoard
from keras.optimizers import RMSprop
from keras import regularizers
from keras.layers import Conv1D, MaxPooling1D, Flatten


In [31]:
from google.colab import files

upload = files.upload()



Saving Data_Cortex_Nuclear .csv to Data_Cortex_Nuclear  (2).csv


In [0]:
df = pd.read_csv('Data_Cortex_Nuclear .csv')

In [0]:
#Impute missing protein expression values with a random value chosen from a normal distribution centered on the protein's mean
#expression value

for c in range(1, (len(df.columns.values))-4):
    df.iloc[:, c].fillna(np.random.normal(np.mean(df.iloc[:, c]), np.std(df.iloc[:, c])), inplace = True)

In [0]:
#Change outcome variables to numeric

df.Genotype.replace({'Control': 0, 'Ts65Dn': 1}, inplace = True)
df.Treatment.replace({'Memantine': 1, 'Saline': 0}, inplace = True)
df.Behavior.replace({'C/S': 0, 'S/C': 1}, inplace = True)
df['class'].replace({'c-CS-m': 0, 'c-SC-m': 1, 'c-CS-s': 2, 'c-SC-s': 3, 't-CS-m': 4, 't-SC-m': 5, 't-CS-s': 6, 't-SC-s': 7}, inplace = True)

In [111]:
df = df.drop(columns = ['MouseID'], axis = 1)
df

,DYRK1A_N,ITSN1_N,BDNF_N,NR1_N,NR2A_N,pAKT_N,pBRAF_N,pCAMKII_N,pCREB_N,pELK_N,...,pCFOS_N,SYP_N,H3AcK18_N,EGR1_N,H3MeK4_N,CaNA_N,Genotype,Treatment,Behavior,class
0,0.503644,0.747193,0.430175,2.816329,5.990152,0.218830,0.177565,2.373744,0.232224,1.750936,...,0.108336,0.427099,0.114783,0.131790,0.128186,1.675652,0,1,0,0
1,0.514617,0.689064,0.411770,2.789514,5.685038,0.211636,0.172817,2.292150,0.226972,1.596377,...,0.104315,0.441581,0.111974,0.135103,0.131119,1.743610,0,1,0,0
2,0.509183,0.730247,0.418309,2.687201,5.622059,0.209011,0.175722,2.283337,0.230247,1.561316,...,0.106219,0.435777,0.111883,0.133362,0.127431,1.926427,0,1,0,0
3,0.442107,0.617076,0.358626,2.466947,4.979503,0.222886,0.176463,2.152301,0.207004,1.595086,...,0.111262,0.391691,0.130405,0.147444,0.146901,1.700563,0,1,0,0
4,0.434940,0.617430,0.358802,2.365785,4.718679,0.213106,0.173627,2.134014,0.192158,1.504230,...,0.110694,0.434154,0.118481,0.140314,0.148380,1.839730,0,1,0,0
5,0.447506,0.628176,0.367388,2.385939,4.807635,0.218578,0.176233,2.141282,0.195188,1.442398,...,0.109446,0.439833,0.116657,0.140766,0.142180,1.816389,0,1,0,0
6,0.428033,0.573696,0.342709,2.334224,4.473130,0.225173,0.184004,2.012414,0.195789,1.612036,...,0.111483,0.406293,0.137229,0.156682,0.157499,1.528484,0,1,0,0
7,0.416923,0.564036,0.327703,2.260135,4.268735,0.214834,0.179668,2.007985,0.189803,1.424601,...,0.113080,0.382532,0.123376,0.155612,0.159409,1.652658,0,1,0,0
8,0.386311,0.538428,0.317720,2.125725,4.063950,0.207222,0.167778,1.861514,0.180684,1.261891,...,0.105490,0.435687,0.125173,0.152711,0.158235,1.834599,0,1,0,0
9,0.380827,0.499294,0.362462,2.096266,3.598587,0.227649,0.188093,1.717861,0.188093,1.414329,...,0.116906,0.414793,0.146133,0.188849,0.192334,1.516862,0,1,0,0


In [115]:
np.unique(df['class'].values)

array([0, 1, 2, 3, 4, 5, 6, 7])

In [0]:
#Create train and test sets for each outcome variable (genotype, learning state, and presence of memantine), as well as for the 'class' variable

xtrains = []
xtests = []
ytrains = []
ytests = []

for y in range(-4,0):
    X_train, X_test, y_train, y_test = train_test_split(np.concatenate([df.iloc[:, :y], df.iloc[:, y:]], axis = 1),df.iloc[:, y], test_size=0.40, random_state=42)
    y_train = keras.utils.to_categorical(y_train, len(np.unique(y_train)))
    y_test = keras.utils.to_categorical(y_test, len(np.unique(y_test)))
    xtrains.append(X_train)
    xtests.append(X_test)
    ytrains.append(y_train)
    ytests.append(y_test)

In [153]:
ytests[2].shape

(432, 2)

In [0]:
#Define a sequential neural network function

def gene_exp_network(X_train, y_train, X_test, y_test):
  model = Sequential()
  model.add(Dense(64, activation = 'relu', input_shape = (X_train.shape[1],)))
  model.add(Dropout(0.5))
  model.add(Dense(32, activation = 'relu'))
  model.add(Dropout(0.5))
  model.add(Dense(10, activation = 'relu'))

  model.add(Dense(y_test.shape[1], activation = 'sigmoid'))

  model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

  history = model.fit(X_train, y_train,
                    batch_size= 30,
                    epochs=100,
                    verbose=False,
                    validation_data=(X_test, y_test))
  score = model.evaluate(X_test, y_test, verbose=0)
  loss = score[0]
  accuracy = score[1]
  return history, loss, accuracy

In [0]:
results = []

for i in range(4):
  network = gene_exp_network(xtrains[i], ytrains[i], xtests[i], ytests[i])
  results.append(network)

In [160]:
np.shape(results)

(4, 3)

In [162]:
print('genotype loss: {}'.format(results[0][1]))
print('genotype accuracy: {}\n'.format(results[0][2]))

print('treatment loss: {}'.format(results[1][1]))
print('treatment accuracy: {}\n'.format(results[1][2]))

print('behavior loss: {}'.format(results[2][1]))
print('behavior accuracy: {}\n'.format(results[2][2]))

print('class loss: {}'.format(results[3][1]))
print('class accuracy: {}'.format(results[3][2]))
      

genotype loss: 3.924819909402517e-07
genotype accuracy: 1.0

treatment loss: 1.6368171895077534e-06
treatment accuracy: 1.0

behavior loss: 8.744062961762177e-07
behavior accuracy: 1.0

class loss: 0.004605673391509939
class accuracy: 1.0
